In [5]:
import senet
import os
import numpy as np
import torch
from torchvision.datasets import ImageFolder
from utils import TransformImage
import shutil
import time

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tensorboardX import SummaryWriter

DATA_DIR = "/home/krf/dataset/BALL/"
traindir = DATA_DIR + "train"
valdir = DATA_DIR +"val"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
BATCH_SIZE = 2
WORKERS = 4
EPOCHS = 20
PRINT_FREQ = 10

In [6]:
model =  senet.se_resnext50_32x4d(num_classes = 2)
#通过随机变化来进行数据增强
train_tf  = TransformImage(
    model,
    scale=1,
    random_crop=False,
    random_hflip=True,
    random_vflip=True,
    random_rotate=True,
    preserve_aspect_ratio=True
)
train_loader = torch.utils.data.DataLoader(
#     datasets.ImageFolder(traindir, transforms.Compose([
# #         transforms.RandomSizedCrop(max(model.input_size)),
#         transforms.RandomHorizontalFlip(),
#         transforms.ToTensor(),
#         normalize,
#     ])),
    datasets.ImageFolder(traindir,train_tf),
    batch_size=BATCH_SIZE, shuffle=True,
    num_workers=WORKERS, pin_memory=True)


val_tf = TransformImage(
    model,
    scale=1,
    preserve_aspect_ratio=True)
val_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(valdir,val_tf),
    batch_size=BATCH_SIZE, shuffle=False,
    num_workers=WORKERS, pin_memory=True)


In [7]:
def train(train_loader, model, criterion, optimizer, epoch,scheduler):
    # switch to train mode
    model.train()
#     end = time.time()
    for i, (input, target) in enumerate(train_loader):
        # measure data loading time
#         data_time.update(time.time() - end)

        target = target.cuda(async=True)
        input_var = torch.autograd.Variable(input.float())
        target_var = torch.autograd.Variable(target)

        # compute output
        output = model(input_var)
        loss = criterion(output, target_var)
        #print(output.data)
        # TP    predict 和 label 同时为1
#         _, pred = output.topk(1, 1, True, True)
#         pred = pred.t()
#         print(pred,target.data)
#         #correct = pred.eq(target.view(1, -1).expand_as(pred))
#         TP += ((pred == 1) & (target.data == 1)).cpu().numpy().sum()
#         # TN    predict 和 label 同时为0
#         TN += ((pred == 0) & (target.data == 0)).cpu().numpy().sum()
#         # FN    predict 0 label 1
#         FN += ((pred == 0) & (target.data == 1)).cpu().numpy().sum()
#         # FP    predict 1 label 0
#         FP += ((pred == 1) & (target.data == 0)).cpu().numpy().sum()
#         print(TP,FP,TN,FN)
        
#         P = TP / (TP + FP)
#         #print(P)
#         R = TP / (TP + FN)
#         F1 = 2 * R * P / (R + P)
#         Acc = (TP + TN) / (TP + TN + FP + FN)
        #print(F1)               
#         # measure accuracy and record loss
#         prec1= accuracy(output.data, target)
#         losses.update(loss.data[0], input.size(0))
#         top1.update(prec1[0], input.size(0))


#         # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

#         # measure elapsed time
#         batch_time.update(time.time() - end)
#         end = time.time()
        meters = trainMeter.update(output,target,loss,input.size(0))

        if i % PRINT_FREQ == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Loss {loss:.5f}\t'
                  'Acc {Acc:.5f}\t'
                  'Precision {P:.5f}\t'
                  'Recall {R:.5f}\t'
                  'F1 {F1:.5f}'.format(
                   epoch,i, len(train_loader), loss=meters[4],
                   Acc=meters[3],P=meters[0],R=meters[1],F1=meters[2]))
            
            step = epoch*len(train_loader) + i
           
            writer.add_scalar('TRAIN/Precision', meters[0], step)
            writer.add_scalar('TRAIN/Recall', meters[1], step)
            writer.add_scalar('TRAIN/F1', meters[2], step)
            writer.add_scalar('TRAIN/Acc', meters[3], step)
            writer.add_scalar('TRAIN/loss',meters[4], step)
            
    scheduler.step(meters[4])


def validate(val_loader, model, criterion,epoch):
    # switch to evaluate mode
    model.eval()
    
#     end = time.time()
    meters = []
    for i, (input, target) in enumerate(val_loader):
        target = target.cuda()
        input = input.cuda()
        input_var = torch.autograd.Variable(input, volatile=True)
        target_var = torch.autograd.Variable(target, volatile=True)
        # compute output
        output = model(input_var)
        loss = criterion(output, target_var)
        meters = valMeter.update(output,target,loss,input.size(0))
        if i % PRINT_FREQ == 0:
            print('Test: [{0}/{1}]\t'
                  'Loss {loss:.5f}\t'
                  'Acc {Acc:.5f}\t'
                  'Precision {P:.5f}\t'
                  'Recall {R:.5f}\t'
                  'F1 {F1:.5f}'.format(
                   i, len(val_loader), loss=meters[4],
                   Acc=meters[3],P=meters[0],R=meters[1],F1=meters[2]))
            
            step = epoch * len(val_loader) + i
            writer.add_scalar('VAL/Precision', meters[0], step)
            writer.add_scalar('VAL/Recall', meters[1], step)
            writer.add_scalar('VAL/F1', meters[2], step)
            writer.add_scalar('VAL/Acc', meters[3], step)
            writer.add_scalar('VAL/loss',meters[4], step)
    print(' * Acc {Acc:.5f} F1 {F1:.5f}'
          .format(Acc=meters[3],F1=meters[2]))
    writer.add_scalar('VAL/EPOCH_F1', meters[2], epoch)
    return meters[2]


def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

class ModelMeter(object):
    def __init__(self):
        self.reset()
        
    def reset(self):
        self.losses = AverageMeter()
        self.top1 = AverageMeter()
        self.TP = 0
        self.TN = 0
        self.FN = 0
        self.FP = 0
        self.P=0
        self.R=0
        self.F1=0
        self.Acc=0

    def update(self, output,target,loss, n=1):
        _, pred = output.data.topk(1, 1, True, True)
        pred = pred.t()
        #print(pred,target.data)
        # TP    predict 和 label 同时为1
        self.TP += ((pred == 1) & (target.data == 1)).cpu().numpy().sum()
        # TN    predict 和 label 同时为0
        self.TN += ((pred == 0) & (target.data == 0)).cpu().numpy().sum()
        # FN    predict 0 label 1
        self.FN += ((pred == 0) & (target.data == 1)).cpu().numpy().sum()
        # FP    predict 1 label 0
        self.FP += ((pred == 1) & (target.data == 0)).cpu().numpy().sum()
        #print(self.TP,self.TN,self.FN,self.FP)
        self.P = self.TP / (self.TP + self.FP)
        self.R = self.TP / (self.TP + self.FN)
        self.F1 = 2 * self.R * self.P / (self.R + self.P)
        
        self.Acc = (self.TP + self.TN) / (self.TP + self.TN + self.FP + self.FN)
        
        self.losses.update(loss.data[0],n)

        return [self.P,self.R,self.F1,self.Acc,self.losses.avg]

class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


# def adjust_learning_rate(optimizer, epoch):
#     """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
#     lr = args.lr * (0.1 ** (epoch // 30))
#     for param_group in optimizer.param_groups:
#         param_group['lr'] = lr


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

In [ ]:
criterion = nn.CrossEntropyLoss().cuda()

optimizer = torch.optim.Adam(model.parameters(), lr = 1e-5)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=2, verbose=True)

#model = model.cuda()
model = torch.nn.DataParallel(model).cuda()
# TP = 0,TN = 0,FN = 0, FP = 0
writer = SummaryWriter()
best_f1 = 0
trainMeter = ModelMeter()
valMeter = ModelMeter()
for epoch in range(EPOCHS):
    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch, scheduler)
    # evaluate on validation set
    F1 = validate(val_loader, model, criterion,epoch)
    
    # remember best prec@1 and save checkpoint
    is_best = F1 > best_f1
    best_f1 = max(F1, best_f1)
    save_checkpoint({
        'epoch': epoch + 1,
        'arch': "SE_ResNeXt50",
        'state_dict': model.state_dict(),
        'best_prec1': best_f1,
    }, is_best)
# export scalar data to JSON for external processing
writer.export_scalars_to_json("./test.json")
writer.close()
    

/home/krf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:151: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch: [0][0/4443]	Loss 0.57152	Acc 1.00000	Precision 1.00000	Recall 1.00000	F1 1.00000
Epoch: [0][10/4443]	Loss 0.75224	Acc 0.77273	Precision 0.50000	Recall 0.20000	F1 0.28571
Epoch: [0][20/4443]	Loss 0.78896	Acc 0.71429	Precision 0.33333	Recall 0.09091	F1 0.14286
Epoch: [0][30/4443]	Loss 0.75184	Acc 0.66129	Precision 0.47059	Recall 0.40000	F1 0.43243
Epoch: [0][40/4443]	Loss 0.67837	Acc 0.70732	Precision 0.52632	Recall 0.40000	F1 0.45455
Epoch: [0][50/4443]	Loss 0.67133	Acc 0.69608	Precision 0.52632	Recall 0.31250	F1 0.39216
Epoch: [0][60/4443]	Loss 0.65505	Acc 0.71311	Precision 0.55000	Recall 0.29730	F1 0.38596
Epoch: [0][70/4443]	Loss 0.62662	Acc 0.72535	Precision 0.55000	Recall 0.26829	F1 0.36066
Epoch: [0][80/4443]	Loss 0.62844	Acc 0.72840	Precision 0.55000	Recall 0.23913	F1 0.33333
Epoch: [0][90/4443]	Loss 0.59955	Acc 0.74725	Precision 0.55000	Recall 0.22917	F1 0.32353
Epoch: [0][100/4443]	Loss 0.60325	Acc 0.74257	Precision 0.55000	Recall 0.20370	F1 0.29730
Epoch: [0][110/4443]	

Epoch: [0][920/4443]	Loss 0.53326	Acc 0.75081	Precision 0.62500	Recall 0.46181	F1 0.53115
Epoch: [0][930/4443]	Loss 0.53337	Acc 0.75081	Precision 0.62530	Recall 0.46046	F1 0.53036
Epoch: [0][940/4443]	Loss 0.53188	Acc 0.75186	Precision 0.62911	Recall 0.46447	F1 0.53440
Epoch: [0][950/4443]	Loss 0.53126	Acc 0.75289	Precision 0.63218	Recall 0.47009	F1 0.53922
Epoch: [0][960/4443]	Loss 0.53052	Acc 0.75338	Precision 0.63265	Recall 0.47208	F1 0.54070
Epoch: [0][970/4443]	Loss 0.53145	Acc 0.75283	Precision 0.63252	Recall 0.47412	F1 0.54198
Epoch: [0][980/4443]	Loss 0.52963	Acc 0.75382	Precision 0.63297	Recall 0.47682	F1 0.54391
Epoch: [0][990/4443]	Loss 0.53342	Acc 0.75227	Precision 0.63181	Recall 0.47386	F1 0.54155
Epoch: [0][1000/4443]	Loss 0.53394	Acc 0.75275	Precision 0.63248	Recall 0.47819	F1 0.54462
Epoch: [0][1010/4443]	Loss 0.53440	Acc 0.75272	Precision 0.63270	Recall 0.47680	F1 0.54380
Epoch: [0][1020/4443]	Loss 0.53592	Acc 0.75220	Precision 0.63235	Recall 0.47627	F1 0.54332
Epoch: 